# Download with tf.datasets

In [ ]:
!pip install tensorflow-datasets > /dev/null

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 162kB/s 
     |████████████████████████████████| 983kB 39.4MB/s 
     |████████████████████████████████| 266kB 42.9MB/s 
     |████████████████████████████████| 1.3MB 37.8MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 471kB 34.1MB/s 
     |████████████████████████████████| 2.9MB 43.2MB/s 
     |████████████████████████████████| 890kB 35.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.1-cp36-none-any.whl size=25276126 sha256=ce9ea56a2edaffeb6c5871d892a626da5315af0e756724b16a110c81be5785f9
  Stored in directory: /root/.cache/pip/wheels/4c/af/2a/3b4aa379166bc910a4132ecb83f795c854b2a4293f83f0b736
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=cf8f1f7cfe6a60c2cea1926669000c9280b2481952cb31ba6f3ba42898b91727
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

# 0. Load dataset for preprocessing

In [ ]:
import tensorflow_datasets as tfds
import ktrain
from ktrain import text as txt 
import numpy as np   
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
(ds_train, ds_test),ds_info = tfds.load(
    name="imdb_reviews",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

INFO:absl:No config specified, defaulting to first: imdb_reviews/plain_text
INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
INFO:absl:Load dataset info from /tmp/tmpwfb06dm5tfds
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset imdb_reviews (/root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0)


INFO:absl:Downloading http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz into /root/tensorflow_datasets/downloads/ai.stanfor.edu_amaas_sentime_aclImdb_v1PaujRp-TxjBWz59jHXsMDm5WiexbxzaFQkEnXc3Tvo8.tar.gz.tmp.0f7022a470824bbb848050b85a07b0d9...
INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-train.tfrecord. Shard lengths: [25000]
INFO:absl:Generating split test


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-test.tfrecord. Shard lengths: [25000]
INFO:absl:Generating split unsupervised


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-unsupervised.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXPRNC0/imdb_reviews-unsupervised.tfrecord. Shard lengths: [50000]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.SKIP.
INFO:absl:Constructing tf.data.Dataset for split ['train', 'test'], from /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
df_train = tfds.as_dataframe(ds_train, ds_info)
df_test = tfds.as_dataframe(ds_test, ds_info)
df = df_train.append(df_test, ignore_index=True)
df['text'] = df['text'].str.decode("utf-8")
df['len'] =  df['text'].str.split().str.len()
df['neg'] = np.where(df['label']==0,1,0)
df['pos'] = np.where(df['label']==1,1,0)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# 4. fastText model

In [ ]:
# preprocessing data
train_fasttext, valid_fasttext, preproc_fasttext = txt.texts_from_df(train_df,
                                                                    text_column='text',
                                                                    label_columns=['neg', 'pos'],
                                                                    ngram_range=2,
                                                                    preprocess_mode='standard',
                                                                    val_pct=0.1)

language: en
Word Counts: 112891
Nrows: 40500
40500 train sequences
train sequence lengths:
	mean : 228
	95percentile : 580
	99percentile : 887
Adding 2-gram features
max_features changed to 1684291 with addition of ngrams
Average train sequence length with ngrams: 455
train (w/ngrams) sequence lengths:
	mean : 455
	95percentile : 1159
	99percentile : 1773
x_train shape: (40500,400)
y_train shape: (40500, 2)
Is Multi-Label? False
4500 test sequences
test sequence lengths:
	mean : 227
	95percentile : 580
	99percentile : 861
Average test sequence length with ngrams: 426
test (w/ngrams) sequence lengths:
	mean : 427
	95percentile : 1081
	99percentile : 1592
x_test shape: (4500,400)
y_test shape: (4500, 2)


In [ ]:
number_of_epochs = 2
batch_size = 8

# Load a fastText model
model_fasttext = txt.text_classifier('fasttext', 
                                    train_fasttext, 
                                    preproc=preproc_fasttext)

# Ktrain learner
learner = ktrain.get_learner(model_fasttext, 
                             train_data=train_fasttext, 
                             val_data=valid_fasttext,
                             batch_size=batch_size)

Is Multi-Label? False
compiling word ID features...
maxlen is 400
done.


In [ ]:
history_fasttext = learner.autofit(0.005, number_of_epochs)



begin training using triangular learning rate policy with max lr of 0.005...
Epoch 1/2
 777/5063 [===>..........................] - ETA: 1:15:54 - loss: 0.8115 - accuracy: 0.5380

In [ ]:
predictor_fasttext = ktrain.get_predictor(learner.model, preproc_fasttext)


1. Accuracy

*   Train: 93%
*   Test: 85%

2. Motivation

*   What in this model's structure makes it suited for representing sentiment?



*   How well you evaluate the accuracy you got for each model?



*   Discuss the main advantages and disadvantages the model has in the particular modelling task.



3. Consistency

*   What you have learned from your analyses.



*   Also, analyze when the model is good and when and where it does not predict well.

# 5. BERT based model 

In [ ]:
# preprocessing data
train_bert, valid_bert, preproc_bert = txt.texts_from_df(train_df,
                                                        'text', # name of column containing review text
                                                        label_columns=['neg', 'pos'],
                                                        maxlen=300,
                                                        preprocess_mode='bert',
                                                        val_pct=0.1)

In [ ]:
batch_size = 8
number_of_epochs=2

# load model
model_bert = txt.text_classifier('bert', train_bert, preproc=preproc_bert)

# wrap model and data in ktrain.Learner object
learner_bert = ktrain.get_learner(model_bert, 
                             train_data=train_bert, 
                             val_data=valid_bert, 
                             batch_size=batch_size) # any batch_size bigger than 10 will result in resource exhaustion on collab

# train model
history_bert = learner_bert.autofit(2e-5, number_of_epochs)

In [ ]:
predictor_bert = ktrain.get_predictor(learner_bert.model, preproc_bert)

1. Accuracy

*   Train: 93%
*   Test: 85%

2. Motivation

*   What in this model's structure makes it suited for representing sentiment?



*   How well you evaluate the accuracy you got for each model?



*   Discuss the main advantages and disadvantages the model has in the particular modelling task.



3. Consistency

*   What you have learned from your analyses.



*   Also, analyze when the model is good and when and where it does not predict well.